In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

In [ ]:
# Data from sde_simulation.ipynb
df_res = pd.read_csv('./train_data/train_res.csv')
df_sm = pd.read_csv('./train_data/train_sm.csv')
df_lb = pd.read_csv('./train_data/train_lb.csv')

In [ ]:
# Shuffle the training set
df_res = df_res.sample(frac=1, axis=1, random_state=999)
df_sm = df_sm.sample(frac=1, axis=1, random_state=999)
df_lb = df_lb.sample(frac=1, random_state=999).reset_index(drop=True)

In [ ]:
n_steps_in, n_steps_out, n_features = 500, 1, 2

In [ ]:
def reshape_inputs(df1, df2, n_features=2):
    # Ensure df1 and df2 are in the correct shape and convert them to NumPy arrays
    arr1 = df1.values.T.reshape(-1, df1.shape[0], 1)  # Shape: (n_samples, n_time_steps, 1)
    arr2 = df2.values.T.reshape(-1, df2.shape[0], 1)  # Shape: (n_samples, n_time_steps, 1)
    
    # Stack the arrays along the last dimension to combine the features
    X = np.concatenate((arr1, arr2), axis=2)  # Shape: (n_samples, n_time_steps, n_features)
    
    return X

In [ ]:
inputs = reshape_inputs(df_res, df_sm)

In [ ]:
inputs.shape

In [ ]:
targets = np.array(df_lb.values.tolist())

In [ ]:
targets.shape

In [ ]:
pool_size_param = 2
learning_rate_param = 0.0005
batch_param = 1024
dropout_percent = 0.1
filters_param = 50
mem_cells = 50
mem_cells2 = 10
kernel_size_param = 12
epoch_param = 300
initializer_param = 'lecun_normal'

In [ ]:
# Model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(filters=filters_param, kernel_size=kernel_size_param, padding='same', activation='relu', input_shape=(n_steps_in, n_features), kernel_initializer=initializer_param))
model.add(tf.keras.layers.Dropout(rate=dropout_percent))
model.add(tf.keras.layers.MaxPool1D(pool_size=pool_size_param))
model.add(tf.keras.layers.LSTM(mem_cells, return_sequences=True, kernel_initializer=initializer_param))
model.add(tf.keras.layers.Dropout(rate=dropout_percent))
model.add(tf.keras.layers.LSTM(mem_cells2, kernel_initializer=initializer_param))
model.add(tf.keras.layers.Dropout(rate=dropout_percent))
model.add(tf.keras.layers.Dense(n_steps_out, activation='sigmoid', kernel_initializer=initializer_param))
opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate_param)
model.compile(optimizer=opt, loss='binary_crossentropy')

In [ ]:
model.summary()

In [ ]:
# Fit model
model.fit(inputs, targets, epochs=epoch_param, batch_size=batch_param, verbose=2)

In [ ]:
model.save('./trained_models/lstm_400ep_1024batch_updated4.h5')